In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

#### SparseCategoricalCrossentropy관련 설명 : https://jins-sw.tistory.com/16

In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 39s]
val_accuracy: 0.8748000264167786

Best val_accuracy So Far: 0.8871999979019165
Total elapsed time: 00h 06m 20s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 192 and the optimal learning rate for the optimizer
is 0.001.



In [9]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.6141 - accuracy: 0.7858 - val_loss: 0.4243 - val_accuracy: 0.8470
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3814 - accuracy: 0.8634 - val_loss: 0.4336 - val_accuracy: 0.8413
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3343 - accuracy: 0.8776 - val_loss: 0.3904 - val_accuracy: 0.8596
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2996 - accuracy: 0.8901 - val_loss: 0.3369 - val_accuracy: 0.8809
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2844 - accuracy: 0.8959 - val_loss: 0.3399 - val_accuracy: 0.8772
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2710 - accuracy: 0.9010 - val_loss: 0.3331 - val_accuracy: 0.8848
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2606 - accuracy: 0.9039 - val_loss: 0.3356 - val_accuracy: